# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and 'alfa' in nombre and "queretaro" in nombre 
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m2_total':'metros_total','m_construido':'metros_construido','m2_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','baños':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','precio_m2_cons':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','Tipo','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38611 entries, 0 to 38610
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    38611 non-null  int64  
 1   Tipo                  38611 non-null  object 
 2   categoria             38611 non-null  object 
 3   propiedad             38611 non-null  object 
 4   precio                38611 non-null  int64  
 5   metros_total          38611 non-null  int64  
 6   metros_construido     38611 non-null  int64  
 7   precio_m2_construido  38611 non-null  float64
 8   estacionamiento       38611 non-null  int64  
 9   recamaras             38611 non-null  int64  
 10  baño                  38611 non-null  int64  
 11  medio_baño            38611 non-null  int64  
 12  baño_total            38611 non-null  float64
dtypes: float64(2), int64(8), object(3)
memory usage: 3.8+ MB


In [23]:
# Definición de rangos de precio
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio <= limite_superior:  # Cambiado a <=
            return categoria
    return None  # Retornar None para manejarlo después

# Limpieza del DataFrame
df_total = df_total.drop(columns=['categoria'], errors='ignore')

# Convertir 'precio' a numérico y manejar errores
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
df_total = df_total.dropna(subset=['precio'])  # Eliminar filas con precios NaN

# Asignar categorías
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

# Verificar categorías inválidas
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])

In [24]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 

# Limitar los valores  
df_total['baño_total'] = df_total['baño_total'].clip(lower=1) #,upper=10
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['metros_total'] != 0]
df_total = df_total[df_total['metros_construido'] != 0]

#df_total = df_total[~df_total['categoria'].str.contains('L')]

In [25]:
df_total.loc[:, 'segmento'] = df_total['categoria'].apply(lambda x: 'ELITE' if x == 'ELITE' else x[0])

In [71]:

df_total = df_total[df_total['metros_total'] != 2500000]
df_total = df_total[df_total['metros_total'] != 290000]
df_total = df_total[df_total['metros_total'] != 11267]


 

In [72]:
df_total['Tipo'].unique()

array(['Casa', 'Departamento', 'Casa En Fraccionamiento', 'Duplex',
       'Condominio Horizontal', 'Casa En Condominio', 'Casa Duplex'],
      dtype=object)

In [73]:
casa_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
depa_df = df_total[df_total['Tipo'].isin(['Departamento', 'Duplex', 'Condominio Horizontal'])]

# Casas

### Revisión de m2

In [70]:
printtt = df_total[df_total['metros_total'] == 11267]
printtt


,id,Tipo,propiedad,precio,metros_total,metros_construido,precio_m2_construido,estacionamiento,recamaras,baño,medio_baño,baño_total,segmento,categoria


In [74]:
casa_df = casa_df[casa_df['metros_total'] > 40]
casa_df = casa_df[casa_df['metros_construido'] > 40]


In [75]:
# metros_total en casas
precios = casa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([   42,    44,    45,    47,    48,    49,    50,    52,    54,
          55,    56,    57,    59,    60,    61,    62,    63,    64,
          65,    66,    67,    68,    69,    70,    71,    72,    73,
          74,    75,    76,    77,    78,    79,    80,    81,    82,
          83,    84,    85,    86,    87,    88,    89,    90,    91,
          92,    93,    94,    95,    96,    97,    98,    99,   100,
         101,   102,   103,   104,   105,   106,   107,   108,   109,
         110,   111,   112,   113,   114,   115,   116,   117,   118,
         119,   120,   121,   122,   123,   124,   125,   126,   127,
         128,   129,   130,   131,   132,   133,   134,   135,   136,
         137,   138,   139,   140,   141,   142,   143,   144,   145,
         146,   147,   148,   149,   150,   151,   152,   153,   154,
         155,   156,   157,   158,   159,   160,   161,   162,   163,
         164,   165,   166,   167,   168,   169,   170,   171,   172,
         173,   174,

In [76]:
# metros_construido en casas
precios = casa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([   41,    42,    44,    45,    47,    48,    49,    50,    51,
          52,    53,    54,    55,    56,    57,    59,    60,    61,
          62,    63,    64,    65,    66,    67,    68,    69,    70,
          71,    72,    73,    74,    75,    76,    77,    78,    79,
          80,    81,    82,    83,    84,    85,    86,    87,    88,
          89,    90,    91,    92,    93,    94,    95,    96,    97,
          98,    99,   100,   101,   102,   103,   104,   105,   106,
         107,   108,   109,   110,   111,   112,   113,   114,   115,
         116,   117,   118,   119,   120,   121,   122,   123,   124,
         125,   126,   127,   128,   129,   130,   131,   132,   133,
         134,   135,   136,   137,   138,   139,   140,   141,   142,
         143,   144,   145,   146,   147,   148,   149,   150,   151,
         152,   153,   154,   155,   156,   157,   158,   159,   160,
         161,   162,   163,   164,   165,   166,   167,   168,   169,
         170,   171,

In [77]:
# metros_total en casas cuando segmento es 'B'
#precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].unique()
precios = casa_df[casa_df['categoria'] == 'B']['metros_total'].value_counts()
#precios = np.sort(precios)
precios

Series([], Name: count, dtype: int64)

## Casa promedio

In [78]:
grupo_promedios = round(casa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
ELITE                  3.0        4.0    3.0
L                      3.0        4.0    3.0
S                      2.0        3.0    3.0
general                2.0        3.0    2.0


## Precio Mercado

In [79]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L','ELITE' ]#,
grupo_descripcion_casa = casa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,290000.0,1.000000e+06,7.588482e+05
1,D,1003000.0,1.750000e+06,1.418873e+06
2,C,1755000.0,2.500000e+06,2.159826e+06
3,B,2521000.0,3.250000e+06,2.901580e+06
4,A,3251000.0,4.000000e+06,3.657923e+06
5,S,4010000.0,1.200000e+07,6.405812e+06
6,L,12220000.0,2.200000e+07,1.676065e+07
7,ELITE,22400000.0,9.680000e+09,4.211759e+07


In [80]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,251,000.00
1     $2,521,000.00
2     $1,755,000.00
3     $1,003,000.00
4       $290,000.00
5    $22,400,000.00
6    $12,220,000.00
7     $4,010,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $4,000,000.00
1        $3,250,000.00
2        $2,500,000.00
3        $1,750,000.00
4        $1,000,000.00
5    $9,680,000,000.00
6       $22,000,000.00
7       $12,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with 

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,251,000.00","$4,000,000.00","$3,657,923.36"
1,B,"$2,521,000.00","$3,250,000.00","$2,901,579.76"
2,C,"$1,755,000.00","$2,500,000.00","$2,159,825.67"
3,D,"$1,003,000.00","$1,750,000.00","$1,418,872.88"
4,E,"$290,000.00","$1,000,000.00","$758,848.18"
5,ELITE,"$22,400,000.00","$9,680,000,000.00","$42,117,586.44"
6,L,"$12,220,000.00","$22,000,000.00","$16,760,654.58"
7,S,"$4,010,000.00","$12,000,000.00","$6,405,812.42"


In [81]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [82]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo             Máximo       Promedio segmento
precio  $290,000.00  $9,680,000,000.00  $6,645,716.63  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $290,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $9,680,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\2555968983.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will 

In [83]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [84]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento          Mínimo             Máximo        Promedio
0        A   $3,251,000.00      $4,000,000.00   $3,657,923.36
1        B   $2,521,000.00      $3,250,000.00   $2,901,579.76
2        C   $1,755,000.00      $2,500,000.00   $2,159,825.67
3        D   $1,003,000.00      $1,750,000.00   $1,418,872.88
4        E     $290,000.00      $1,000,000.00     $758,848.18
5    ELITE  $22,400,000.00  $9,680,000,000.00  $42,117,586.44
6        L  $12,220,000.00     $22,000,000.00  $16,760,654.58
7        S   $4,010,000.00     $12,000,000.00   $6,405,812.42
8  general     $290,000.00  $9,680,000,000.00   $6,645,716.63


In [85]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$290,000.00","$1,000,000.00","$758,848.18"
1,D,"$1,003,000.00","$1,750,000.00","$1,418,872.88"
2,C,"$1,755,000.00","$2,500,000.00","$2,159,825.67"
3,B,"$2,521,000.00","$3,250,000.00","$2,901,579.76"
4,A,"$3,251,000.00","$4,000,000.00","$3,657,923.36"
5,S,"$4,010,000.00","$12,000,000.00","$6,405,812.42"
6,L,"$12,220,000.00","$22,000,000.00","$16,760,654.58"
7,ELITE,"$22,400,000.00","$9,680,000,000.00","$42,117,586.44"
8,general,"$290,000.00","$9,680,000,000.00","$6,645,716.63"


In [86]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [95]:
#casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE']#,
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion_casa.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.0,1300.0,209.641183
1,D,44.0,2144.0,131.902098
2,C,42.0,1000.0,125.571581
3,B,52.0,1387.0,146.866263
4,A,60.0,999.0,177.335589
5,S,50.0,11000.0,296.694746
6,L,103.0,80000.0,899.412825
7,ELITE,70.0,58000.0,1456.817661


In [88]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     60.00 m²
1     52.00 m²
2     42.00 m²
3     44.00 m²
4     42.00 m²
5     70.00 m²
6    103.00 m²
7     50.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       999.00 m²
1     1,387.00 m²
2     1,000.00 m²
3     2,144.00 m²
4     1,300.00 m²
5    58,000.00 m²
6    80,000.00 m²
7    11,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripc

,segmento,Mínimo,Máximo,Promedio
0,A,60.00 m²,999.00 m²,177.34 m²
1,B,52.00 m²,"1,387.00 m²",146.87 m²
2,C,42.00 m²,"1,000.00 m²",125.57 m²
3,D,44.00 m²,"2,144.00 m²",131.90 m²
4,E,42.00 m²,"1,300.00 m²",209.64 m²
5,ELITE,70.00 m²,"58,000.00 m²","1,456.82 m²"
6,L,103.00 m²,"80,000.00 m²",899.41 m²
7,S,50.00 m²,"11,000.00 m²",296.69 m²


In [89]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [94]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
percentil_95 = casa_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo        Máximo   Promedio segmento
metros_total  42.00 m²  35,888.89 m²  318.70 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\1093795092.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\1093795092.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    35,888.89 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17404\1093795092.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and w

In [91]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [92]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo     Promedio
0        A   60.00 m²     999.00 m²    177.34 m²
1        B   52.00 m²   1,387.00 m²    146.87 m²
2        C   42.00 m²   1,000.00 m²    125.57 m²
3        D   44.00 m²   2,144.00 m²    131.90 m²
4        E   42.00 m²   1,300.00 m²    209.64 m²
5    ELITE   70.00 m²  58,000.00 m²  1,456.82 m²
6        L  103.00 m²  80,000.00 m²    899.41 m²
7        S   50.00 m²  11,000.00 m²    296.69 m²
8  general   42.00 m²  80,000.00 m²    318.70 m²


In [93]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','ELITE','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.00 m²,"1,300.00 m²",209.64 m²
1,D,44.00 m²,"2,144.00 m²",131.90 m²
2,C,42.00 m²,"1,000.00 m²",125.57 m²
3,B,52.00 m²,"1,387.00 m²",146.87 m²
4,A,60.00 m²,999.00 m²,177.34 m²
5,S,50.00 m²,"11,000.00 m²",296.69 m²
6,L,103.00 m²,"80,000.00 m²",899.41 m²
7,ELITE,70.00 m²,"58,000.00 m²","1,456.82 m²"
8,general,42.00 m²,"80,000.00 m²",318.70 m²


In [44]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [30]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S','L']#, 
grupo_descripcion_casa = casa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,35.0,280000.0,525.480524
1,D,42.0,1064.0,131.787102
2,C,42.0,11267.0,138.982915
3,B,15.0,543.0,159.501175
4,A,60.0,999.0,194.529050
5,S,53.0,15683.0,304.187943
6,L,180.0,80000.0,646.056704


In [31]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     60.00 m²
1     15.00 m²
2     42.00 m²
3     42.00 m²
4     35.00 m²
5    180.00 m²
6     53.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        999.00 m²
1        543.00 m²
2     11,267.00 m²
3      1,064.00 m²
4    280,000.00 m²
5     80,000.00 m²
6     15,683.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grup

,segmento,Mínimo,Máximo,Promedio
0,A,60.00 m²,999.00 m²,194.53 m²
1,B,15.00 m²,543.00 m²,159.50 m²
2,C,42.00 m²,"11,267.00 m²",138.98 m²
3,D,42.00 m²,"1,064.00 m²",131.79 m²
4,E,35.00 m²,"280,000.00 m²",525.48 m²
5,L,180.00 m²,"80,000.00 m²",646.06 m²
6,S,53.00 m²,"15,683.00 m²",304.19 m²


In [32]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [33]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo         Máximo   Promedio segmento
metros_construido  15.00 m²  280,000.00 m²  301.27 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    15.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    280,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2239152448.py:24: FutureWarning: Setting an item of incompatible dtype is depre

In [34]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [35]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo         Máximo   Promedio
0        A   60.00 m²      999.00 m²  194.53 m²
1        B   15.00 m²      543.00 m²  159.50 m²
2        C   42.00 m²   11,267.00 m²  138.98 m²
3        D   42.00 m²    1,064.00 m²  131.79 m²
4        E   35.00 m²  280,000.00 m²  525.48 m²
5        L  180.00 m²   80,000.00 m²  646.06 m²
6        S   53.00 m²   15,683.00 m²  304.19 m²
7  general   15.00 m²  280,000.00 m²  301.27 m²


In [36]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,35.00 m²,"280,000.00 m²",525.48 m²
1,D,42.00 m²,"1,064.00 m²",131.79 m²
2,C,42.00 m²,"11,267.00 m²",138.98 m²
3,B,15.00 m²,543.00 m²,159.50 m²
4,A,60.00 m²,999.00 m²,194.53 m²
5,S,53.00 m²,"15,683.00 m²",304.19 m²
6,L,180.00 m²,"80,000.00 m²",646.06 m²
7,general,15.00 m²,"280,000.00 m²",301.27 m²


In [37]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [38]:
casa_df['precio_m2_construido']

0        17254.90
1        20746.27
2        18410.01
4         1538.46
5         3950.82
           ...   
38605     1689.81
38606    10588.89
38607    13448.28
38608    20955.56
38609     4736.84
Name: precio_m2_construido, Length: 28496, dtype: float64

In [39]:
casa_df.loc[:, 'segmento'] = casa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L']#
grupo_descripcion_casa = casa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,2.46,31428571.43,30249.484925
1,D,1221.80,28571.43,14411.496146
2,C,164.11,56818.18,17743.751479
3,B,5930.02,175333.33,19094.775277
4,A,3622.02,60000.00,19817.068589
5,S,464.20,111320.75,21842.615834
6,L,237.50,77777.78,31829.821426


In [40]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $3,622.02
1    $5,930.02
2      $164.11
3    $1,221.80
4        $2.46
5      $237.50
6      $464.20
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $60,000.00
1       $175,333.33
2        $56,818.18
3        $28,571.43
4    $31,428,571.43
5        $77,777.78
6       $111,320.75
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = 

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,622.02","$60,000.00","$19,817.07"
1,B,"$5,930.02","$175,333.33","$19,094.78"
2,C,$164.11,"$56,818.18","$17,743.75"
3,D,"$1,221.80","$28,571.43","$14,411.50"
4,E,$2.46,"$31,428,571.43","$30,249.48"
5,L,$237.50,"$77,777.78","$31,829.82"
6,S,$464.20,"$111,320.75","$21,842.62"


In [41]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [42]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = casa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo          Máximo    Promedio segmento
precio_m2_construido  $2.46  $31,428,571.43  $22,168.14  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $2.46
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $31,428,571.43
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\155160619.py:23: FutureWarning: Setting an item of incompatible dtype is depr

In [43]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [44]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo          Máximo    Promedio
0        A  $3,622.02      $60,000.00  $19,817.07
1        B  $5,930.02     $175,333.33  $19,094.78
2        C    $164.11      $56,818.18  $17,743.75
3        D  $1,221.80      $28,571.43  $14,411.50
4        E      $2.46  $31,428,571.43  $30,249.48
5        L    $237.50      $77,777.78  $31,829.82
6        S    $464.20     $111,320.75  $21,842.62
7  general      $2.46  $31,428,571.43  $22,168.14


In [45]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,$2.46,"$31,428,571.43","$30,249.48"
1,D,"$1,221.80","$28,571.43","$14,411.50"
2,C,$164.11,"$56,818.18","$17,743.75"
3,B,"$5,930.02","$175,333.33","$19,094.78"
4,A,"$3,622.02","$60,000.00","$19,817.07"
5,S,$464.20,"$111,320.75","$21,842.62"
6,L,$237.50,"$77,777.78","$31,829.82"
7,general,$2.46,"$31,428,571.43","$22,168.14"


In [46]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_casa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# Depas

### Revisión de m2

In [47]:
# metros_total en casas
precios = depa_df['metros_total'].unique()
precios = np.sort(precios)
precios

array([   30,    42,    44,    45,    46,    48,    49,    50,    51,
          52,    54,    55,    56,    57,    58,    59,    60,    61,
          62,    63,    64,    65,    66,    67,    68,    69,    70,
          71,    72,    73,    74,    75,    76,    77,    78,    79,
          80,    81,    82,    83,    84,    85,    86,    87,    88,
          89,    90,    91,    92,    93,    94,    95,    96,    97,
          98,    99,   100,   101,   102,   103,   104,   105,   106,
         107,   108,   109,   110,   111,   112,   113,   114,   115,
         116,   117,   118,   119,   120,   121,   122,   123,   124,
         125,   126,   127,   128,   129,   130,   131,   132,   133,
         134,   135,   136,   137,   138,   139,   140,   141,   142,
         143,   144,   145,   146,   147,   148,   149,   150,   151,
         152,   153,   154,   155,   156,   157,   158,   159,   160,
         161,   162,   163,   164,   165,   166,   167,   168,   169,
         170,   171,

In [48]:
# metros_construido en casas
precios = depa_df['metros_construido'].unique()
precios = np.sort(precios)
precios

array([   42,    44,    45,    46,    48,    49,    50,    51,    52,
          53,    54,    55,    56,    57,    58,    59,    60,    61,
          62,    63,    64,    65,    66,    67,    68,    69,    70,
          71,    72,    73,    74,    75,    76,    77,    78,    79,
          80,    81,    82,    83,    84,    85,    86,    87,    88,
          89,    90,    91,    92,    93,    94,    95,    96,    97,
          98,    99,   100,   101,   102,   103,   104,   105,   106,
         107,   108,   109,   110,   111,   112,   113,   114,   115,
         116,   117,   118,   119,   120,   121,   122,   123,   124,
         125,   126,   127,   128,   129,   130,   131,   132,   133,
         134,   135,   136,   137,   138,   139,   140,   141,   142,
         143,   144,   145,   146,   147,   148,   149,   150,   151,
         152,   153,   154,   155,   156,   157,   158,   159,   160,
         161,   162,   163,   164,   165,   166,   167,   168,   169,
         170,   171,

In [49]:
# metros_total en casas cuando segmento es 'B'
#precios = casa_df[casa_df['segmento'] == 'B']['metros_total'].unique()
precios = depa_df[depa_df['categoria'] == 'B']['metros_total'].value_counts()
#precios = np.sort(precios)
precios

Series([], Name: count, dtype: int64)

## Depa promedio

In [50]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0]
grupo_promedios = round(depa_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(depa_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

#print(grupo_promedios)
#print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        2.0    2.0
B                      2.0        2.0    2.0
C                      2.0        2.0    2.0
D                      2.0        2.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        2.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\439086565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0]


## Precio Mercado

In [66]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L']
grupo_descripcion_casa = depa_df.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,280000.0,4.450000e+09,7.891834e+07
1,D,1020000.0,1.750000e+06,1.383677e+06
2,C,1752000.0,2.500000e+06,2.194163e+06
3,B,2531600.0,3.250000e+06,2.907469e+06
4,A,3254000.0,4.000000e+06,3.630013e+06
5,S,4003000.0,1.200000e+07,5.698818e+06
6,L,12331000.0,2.100000e+07,1.381341e+07


In [67]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,254,000.00
1     $2,531,600.00
2     $1,752,000.00
3     $1,020,000.00
4       $280,000.00
5    $12,331,000.00
6     $4,003,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $4,000,000.00
1        $3,250,000.00
2        $2,500,000.00
3        $1,750,000.00
4    $4,450,000,000.00
5       $21,000,000.00
6       $12,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compat

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,254,000.00","$4,000,000.00","$3,630,012.62"
1,B,"$2,531,600.00","$3,250,000.00","$2,907,469.22"
2,C,"$1,752,000.00","$2,500,000.00","$2,194,163.44"
3,D,"$1,020,000.00","$1,750,000.00","$1,383,676.86"
4,E,"$280,000.00","$4,450,000,000.00","$78,918,339.85"
5,L,"$12,331,000.00","$21,000,000.00","$13,813,413.44"
6,S,"$4,003,000.00","$12,000,000.00","$5,698,818.28"


In [68]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [69]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

             Mínimo             Máximo       Promedio segmento
precio  $280,000.00  $4,450,000,000.00  $5,946,885.83  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $280,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $4,450,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3749309691.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will 

In [70]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [71]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento          Mínimo             Máximo        Promedio
0        A   $3,254,000.00      $4,000,000.00   $3,630,012.62
1        B   $2,531,600.00      $3,250,000.00   $2,907,469.22
2        C   $1,752,000.00      $2,500,000.00   $2,194,163.44
3        D   $1,020,000.00      $1,750,000.00   $1,383,676.86
4        E     $280,000.00  $4,450,000,000.00  $78,918,339.85
5        L  $12,331,000.00     $21,000,000.00  $13,813,413.44
6        S   $4,003,000.00     $12,000,000.00   $5,698,818.28
7  general     $280,000.00  $4,450,000,000.00   $5,946,885.83


In [72]:
orden_segmentos = ['E', 'D', 'C', 'B','A', 'S', 'L' ,'general']
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$280,000.00","$4,450,000,000.00","$78,918,339.85"
1,D,"$1,020,000.00","$1,750,000.00","$1,383,676.86"
2,C,"$1,752,000.00","$2,500,000.00","$2,194,163.44"
3,B,"$2,531,600.00","$3,250,000.00","$2,907,469.22"
4,A,"$3,254,000.00","$4,000,000.00","$3,630,012.62"
5,S,"$4,003,000.00","$12,000,000.00","$5,698,818.28"
6,L,"$12,331,000.00","$21,000,000.00","$13,813,413.44"
7,general,"$280,000.00","$4,450,000,000.00","$5,946,885.83"


In [58]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Terreno

In [59]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L']
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_total'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.0,2296.0,171.185841
1,D,50.0,860.0,81.405797
2,C,55.0,711.0,103.275568
3,B,55.0,2017.0,109.303878
4,A,30.0,22415.0,153.425520
5,S,52.0,576.0,192.720023
6,L,189.0,958.0,395.581395


In [60]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     30.00 m²
1     55.00 m²
2     55.00 m²
3     50.00 m²
4     42.00 m²
5    189.00 m²
6     52.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    22,415.00 m²
1     2,017.00 m²
2       711.00 m²
3       860.00 m²
4     2,296.00 m²
5       958.00 m²
6       576.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descr

,segmento,Mínimo,Máximo,Promedio
0,A,30.00 m²,"22,415.00 m²",153.43 m²
1,B,55.00 m²,"2,017.00 m²",109.30 m²
2,C,55.00 m²,711.00 m²,103.28 m²
3,D,50.00 m²,860.00 m²,81.41 m²
4,E,42.00 m²,"2,296.00 m²",171.19 m²
5,L,189.00 m²,958.00 m²,395.58 m²
6,S,52.00 m²,576.00 m²,192.72 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [61]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_total'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                Mínimo        Máximo   Promedio segmento
metros_total  30.00 m²  22,415.00 m²  151.31 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    30.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    22,415.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\2326618923.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and w

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [62]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo   Promedio
0        A   30.00 m²  22,415.00 m²  153.43 m²
1        B   55.00 m²   2,017.00 m²  109.30 m²
2        C   55.00 m²     711.00 m²  103.28 m²
3        D   50.00 m²     860.00 m²   81.41 m²
4        E   42.00 m²   2,296.00 m²  171.19 m²
5        L  189.00 m²     958.00 m²  395.58 m²
6        S   52.00 m²     576.00 m²  192.72 m²
7  general   30.00 m²  22,415.00 m²  151.31 m²


In [64]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.00 m²,"2,296.00 m²",171.19 m²
1,D,50.00 m²,860.00 m²,81.41 m²
2,C,55.00 m²,711.00 m²,103.28 m²
3,B,55.00 m²,"2,017.00 m²",109.30 m²
4,A,30.00 m²,"22,415.00 m²",153.43 m²
5,S,52.00 m²,576.00 m²,192.72 m²
6,L,189.00 m²,958.00 m²,395.58 m²
7,general,30.00 m²,"22,415.00 m²",151.31 m²


In [65]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_terreno_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## M2 Construcción

In [73]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L']#
grupo_descripcion_casa = depa_df.groupby('segmento')['metros_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.0,2296.0,170.424779
1,D,44.0,400.0,79.660079
2,C,55.0,711.0,103.642045
3,B,55.0,2017.0,109.766323
4,A,51.0,22415.0,155.374134
5,S,52.0,1210.0,180.668304
6,L,165.0,805.0,383.720930


In [74]:
# Función para formatear valores en formato monetario por m2
def formato_monetario_m2(valor):
    return f"{valor:,.2f} m²"  # Formato con $, 2 decimales, separador de miles y unidad m²

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     51.00 m²
1     55.00 m²
2     55.00 m²
3     44.00 m²
4     42.00 m²
5    165.00 m²
6     52.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario_m2)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3382106320.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    22,415.00 m²
1     2,017.00 m²
2       711.00 m²
3       400.00 m²
4     2,296.00 m²
5       805.00 m²
6     1,210.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descr

,segmento,Mínimo,Máximo,Promedio
0,A,51.00 m²,"22,415.00 m²",155.37 m²
1,B,55.00 m²,"2,017.00 m²",109.77 m²
2,C,55.00 m²,711.00 m²,103.64 m²
3,D,44.00 m²,400.00 m²,79.66 m²
4,E,42.00 m²,"2,296.00 m²",170.42 m²
5,L,165.00 m²,805.00 m²,383.72 m²
6,S,52.00 m²,"1,210.00 m²",180.67 m²


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [75]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['metros_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95


# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"{valor:,.2f} m²"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                     Mínimo        Máximo   Promedio segmento
metros_construido  42.00 m²  22,415.00 m²  146.91 m²  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    22,415.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\4103975992.py:24: FutureWarning: Setting an item of incompatible dtype is deprec

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [76]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento     Mínimo        Máximo   Promedio
0        A   51.00 m²  22,415.00 m²  155.37 m²
1        B   55.00 m²   2,017.00 m²  109.77 m²
2        C   55.00 m²     711.00 m²  103.64 m²
3        D   44.00 m²     400.00 m²   79.66 m²
4        E   42.00 m²   2,296.00 m²  170.42 m²
5        L  165.00 m²     805.00 m²  383.72 m²
6        S   52.00 m²   1,210.00 m²  180.67 m²
7  general   42.00 m²  22,415.00 m²  146.91 m²


In [77]:
orden_segmentos = ['E', 'D', 'C', 'B','A', 'S', 'L', 'general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,42.00 m²,"2,296.00 m²",170.42 m²
1,D,44.00 m²,400.00 m²,79.66 m²
2,C,55.00 m²,711.00 m²,103.64 m²
3,B,55.00 m²,"2,017.00 m²",109.77 m²
4,A,51.00 m²,"22,415.00 m²",155.37 m²
5,S,52.00 m²,"1,210.00 m²",180.67 m²
6,L,165.00 m²,805.00 m²,383.72 m²
7,general,42.00 m²,"22,415.00 m²",146.91 m²


In [78]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

## Pecio M2 Construcción

In [79]:
depa_df['precio_m2_construido']

3         4590.16
14        8884.62
19        8428.57
25       15000.00
27       14130.43
           ...   
36643    53571.43
36644    53571.43
36645    53571.43
36646    53571.43
36647    53571.43
Name: precio_m2_construido, Length: 9005, dtype: float64

In [80]:
depa_df.loc[:, 'segmento'] = depa_df['categoria'].str[0].astype(str)
orden_segmentos = ['E', 'D', 'C', 'B','A', 'S', 'L']# 
grupo_descripcion_casa = depa_df.groupby('segmento')['precio_m2_construido'].describe().reset_index()
grupo_descripcion_casa = grupo_descripcion_casa.loc[:, ['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion_casa.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
# Ordenar el dataframe según el orden específico de segmentos
ordenado = grupo_descripcion_casa.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,2605.56,38695652.17,633001.845265
1,D,3250.00,26461.54,18003.856258
2,C,3516.17,45188.18,22849.048849
3,B,1363.41,50508.47,28069.935852
4,A,169.53,69835.68,29192.648372
5,S,3636.36,88461.54,33510.179474
6,L,22360.25,74733.33,38681.076047


In [81]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion_casa 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       $169.53
1     $1,363.41
2     $3,516.17
3     $3,250.00
4     $2,605.56
5    $22,360.25
6     $3,636.36
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:, i] = grupo_descripcion_casa.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\3321038986.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        $69,835.68
1        $50,508.47
2        $45,188.18
3        $26,461.54
4    $38,695,652.17
5        $74,733.33
6        $88,461.54
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion_casa.iloc[:

,segmento,Mínimo,Máximo,Promedio
0,A,$169.53,"$69,835.68","$29,192.65"
1,B,"$1,363.41","$50,508.47","$28,069.94"
2,C,"$3,516.17","$45,188.18","$22,849.05"
3,D,"$3,250.00","$26,461.54","$18,003.86"
4,E,"$2,605.56","$38,695,652.17","$633,001.85"
5,L,"$22,360.25","$74,733.33","$38,681.08"
6,S,"$3,636.36","$88,461.54","$33,510.18"


In [ ]:
# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
#grupo_descripcion.rename(columns={
#    'min': 'Mínimo',
#    'max': 'Máximo',
#    'mean': 'Promedio',
#}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
#percentiles_95 = casa_df.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
#print(grupo_descripcion)

In [82]:
# Filtrar los datos según las condiciones especificadas
#filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Calcular las estadísticas descriptivas
todo_descripcion = depa_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general' # Agregar la columna de segmento
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Calcular el percentil 95%
#percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)
# Reemplazar el valor máximo con el percentil 95%
#todo_descripcion['Máximo'] = percentil_95

# Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
print(todo_descripcion)

                       Mínimo          Máximo    Promedio segmento
precio_m2_construido  $169.53  $38,695,652.17  $44,403.61  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $169.53
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $38,695,652.17
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_14352\1082635710.py:23: FutureWarning: Setting an item of incompatible dtype is

In [ ]:
# Agrupar por segmento y calcular las estadísticas descriptivas
#grupo_descripcion = casa_df.groupby('segmento')['precio'].describe().reset_index()
#grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']] # Seleccionar las columnas deseadas
#grupo_descripcion.rename(columns={'min': 'Mínimo','max': 'Máximo', 'mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% para cada segmento
#percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
#percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
#grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
#grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
#grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Paso 7: Calcular las estadísticas generales
#general_stats = df_total['precio'].describe()[['min', 'max', 'mean']].to_frame().T
#general_stats.rename(columns={'min': 'Mínimo','max': 'Máximo','mean': 'Promedio',}, inplace=True)

# Calcular el percentil 95% general
#general_percentil_95 = df_total['precio'].quantile(0.95)

# Reemplazar el máximo general con el percentil 95%
#general_stats['Máximo'] = general_percentil_95

# Agregar una fila para los resultados generales
#general_stats['segmento'] = 'General'

# Concatenar los resultados generales al DataFrame de segmentos
#resultado_final = pd.concat([grupo_descripcion, general_stats], ignore_index=True)
#print(resultado_final)

In [83]:
resultado_final = pd.concat([grupo_descripcion_casa, todo_descripcion], ignore_index=True)
print(resultado_final)

  segmento      Mínimo          Máximo     Promedio
0        A     $169.53      $69,835.68   $29,192.65
1        B   $1,363.41      $50,508.47   $28,069.94
2        C   $3,516.17      $45,188.18   $22,849.05
3        D   $3,250.00      $26,461.54   $18,003.86
4        E   $2,605.56  $38,695,652.17  $633,001.85
5        L  $22,360.25      $74,733.33   $38,681.08
6        S   $3,636.36      $88,461.54   $33,510.18
7  general     $169.53  $38,695,652.17   $44,403.61


In [84]:
orden_segmentos = ['E', 'D', 'C', 'B', 'A', 'S', 'L','general']#
ordenado = resultado_final.set_index('segmento').reindex(orden_segmentos).reset_index()
ordenado

,segmento,Mínimo,Máximo,Promedio
0,E,"$2,605.56","$38,695,652.17","$633,001.85"
1,D,"$3,250.00","$26,461.54","$18,003.86"
2,C,"$3,516.17","$45,188.18","$22,849.05"
3,B,"$1,363.41","$50,508.47","$28,069.94"
4,A,$169.53,"$69,835.68","$29,192.65"
5,S,"$3,636.36","$88,461.54","$33,510.18"
6,L,"$22,360.25","$74,733.33","$38,681.08"
7,general,$169.53,"$38,695,652.17","$44,403.61"


In [85]:
# Guardar el JSON en un archivo
resultados_finales_desc_json = resultado_final.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos_depa.json', 'w') as f:
    f.write(resultados_finales_desc_json)

# ______
# antes

In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500001, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),
    "B2": (2750001, 3000000),
    "B3": (3000001, 3250000),
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf')),
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None
df_total = df_total.drop(columns=['categoria'], errors='ignore')


# Convertir 'precio' a numérico y manejar errores para tener la categoría correcta 
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]


#df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=10) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

df_total = df_total[df_total['precio'] != 0]
#df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

In [9]:
df_total['Tipo'].unique()

array(['Casa', 'Departamento', 'Casa En Fraccionamiento', 'Duplex',
       'Condominio Horizontal', 'Casa En Condominio', 'Casa Duplex'],
      dtype=object)

In [10]:
casas_df = df_total[df_total['Tipo'].isin(['Casa', 'Casa En Fraccionamiento', 'Casa En Condominio', 'Casa Duplex'])]
departamentos_df = df_total[df_total['Tipo'].isin(['Departamento','Duplex','Condominio Horizontal'])]

## Casa promedio

In [11]:
casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]
grupo_promedios = round(casas_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(casas_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        4.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        4.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    3.0
general                2.0        3.0    2.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\2149301391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  casas_df.loc[:, 'segmento'] = casas_df['categoria'].str[0]


## Depa promedio

In [13]:
departamentos_df.loc[:, 'segmento'] = departamentos_df['categoria'].str[0]
grupo_promedios = round(departamentos_df.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(departamentos_df[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)

print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)

# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_depa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        2.0    2.0
B                      2.0        2.0    2.0
C                      2.0        2.0    2.0
D                      2.0        2.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          2      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        2.0    2.0
B                      2.0        2.0    2.0
C                      2.0        2.0    2.0
D                      2.0        2.0    2.0
E                      2.0        3.0    2.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        2.0    2.0


## Precio Mercado

In [14]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean']] #Columnas deseadas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio
0,A,3251000.0,3.999999e+06,3.643422e+06
1,B,2521000.0,3.248000e+06,2.894521e+06
2,C,1752000.0,2.499000e+06,2.149352e+06
3,D,1003000.0,1.742360e+06,1.406856e+06
4,E,280000.0,9.680000e+09,2.001560e+07
5,L,12220000.0,2.200000e+07,1.640851e+07
6,S,500000.0,2.200000e+07,6.109972e+06


In [15]:

# Paso 1: Agrupar por segmento y calcular las estadísticas descriptivas
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()

# Paso 2: Seleccionar las columnas deseadas
grupo_descripcion = grupo_descripcion[['segmento', 'min', 'max', 'mean']]

# Paso 3: Renombrar las columnas
grupo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 4: Calcular el percentil 95% para cada segmento
percentiles_95 = df_total.groupby('segmento')['precio'].quantile(0.95).reset_index()
percentiles_95.rename(columns={'precio': 'Máximo_95%'}, inplace=True)

# Paso 5: Unir los percentiles 95% con el DataFrame original
grupo_descripcion = grupo_descripcion.merge(percentiles_95, on='segmento')

# Paso 6: Reemplazar el valor máximo con el percentil 95%
grupo_descripcion['Máximo'] = grupo_descripcion['Máximo_95%']

# Eliminar la columna temporal
grupo_descripcion.drop(columns=['Máximo_95%'], inplace=True)

# Mostrar el resultado final
print(grupo_descripcion)


  segmento      Mínimo      Máximo      Promedio
0        A   3251000.0   3981000.0  3.643422e+06
1        B   2521000.0   3200000.0  2.894521e+06
2        C   1752000.0   2476400.0  2.149352e+06
3        D   1003000.0   1708402.8  1.406856e+06
4        E    280000.0  39300000.0  2.001560e+07
5        L  12220000.0  21200000.0  1.640851e+07
6        S    500000.0  10573000.0  6.109972e+06


In [29]:
# Calculate the average of the 'Mínimo' column for all segments except 'general'
average_minimo = grupo_descripcion[grupo_descripcion['segmento'] != 'general']['Mínimo'].mean()
average_minimo

np.float64(3075285.714285714)

In [16]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,251,000.00
1     $2,521,000.00
2     $1,752,000.00
3     $1,003,000.00
4       $280,000.00
5    $12,220,000.00
6       $500,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1329763181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,981,000.00
1     $3,200,000.00
2     $2,476,400.00
3     $1,708,402.80
4    $39,300,000.00
5    $21,200,000.00
6    $10,573,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descr

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,251,000.00","$3,981,000.00","$3,643,422.26"
1,B,"$2,521,000.00","$3,200,000.00","$2,894,521.41"
2,C,"$1,752,000.00","$2,476,400.00","$2,149,352.02"
3,D,"$1,003,000.00","$1,708,402.80","$1,406,855.54"
4,E,"$280,000.00","$39,300,000.00","$20,015,602.83"
5,L,"$12,220,000.00","$21,200,000.00","$16,408,514.42"
6,S,"$500,000.00","$10,573,000.00","$6,109,971.68"


In [31]:
todo_descripcion = df_total[(~df_total['metros_construido'].isin([0,1,2,3,4,5])) & (df_total['precio_m2_construido'] >= 280000)]['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10660\1880294109.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $284,444.44
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10660\1880294109.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $38,695,652.17
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10660\1880294109.py:12: FutureWarning: Setting an item of incompatible dtyp

,Mínimo,Máximo,Promedio,segmento
precio_m2_construido,"$284,444.44","$38,695,652.17","$13,397,198.33",general


In [17]:
import pandas as pd

# Supongamos que df_total ya está definido y contiene las columnas 'metros_construido' y 'precio_m2_construido'.

# Filtrar los datos según las condiciones especificadas
filtrado_df = df_total[(~df_total['metros_construido'].isin([0, 1, 2, 3, 4, 5])) & (df_total['precio_m2_construido'] >= 280000)]

# Paso 1: Calcular las estadísticas descriptivas
todo_descripcion = filtrado_df['precio_m2_construido'].describe()[['min', 'max', 'mean']].to_frame().T

# Agregar la columna de segmento
todo_descripcion['segmento'] = 'general'

# Paso 2: Renombrar las columnas
todo_descripcion.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Paso 3: Calcular el percentil 95%
percentil_95 = filtrado_df['precio_m2_construido'].quantile(0.95)

# Paso 4: Reemplazar el valor máximo con el percentil 95%
todo_descripcion['Máximo'] = percentil_95

# Paso 5: Aplicar formato monetario a los valores
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato monetario con dos decimales

for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

# Mostrar el resultado final
print(todo_descripcion)


                           Mínimo          Máximo        Promedio segmento
precio_m2_construido  $284,444.44  $35,584,643.85  $13,397,198.33  general


C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $284,444.44
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $35,584,643.85
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\79234579.py:32: FutureWarning: Setting an item of incompatible dtype is d

In [18]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [19]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,"$3,251,000.00","$3,981,000.00","$3,643,422.26"
1,B,"$2,521,000.00","$3,200,000.00","$2,894,521.41"
2,C,"$1,752,000.00","$2,476,400.00","$2,149,352.02"
3,D,"$1,003,000.00","$1,708,402.80","$1,406,855.54"
4,E,"$280,000.00","$39,300,000.00","$20,015,602.83"
5,L,"$12,220,000.00","$21,200,000.00","$16,408,514.42"
6,S,"$500,000.00","$10,573,000.00","$6,109,971.68"
7,general,"$284,444.44","$35,584,643.85","$13,397,198.33"


In [35]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [36]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [36]:
resultados_finales_desc_json

'{"0":{"segmento":"A","M\\u00ednimo":"$169.53","M\\u00e1ximo":"$37,563.22","Promedio":"$23,359.40"},"1":{"segmento":"B","M\\u00ednimo":"$5,930.02","M\\u00e1ximo":"$35,119.05","Promedio":"$22,788.10"},"2":{"segmento":"C","M\\u00ednimo":"$164.11","M\\u00e1ximo":"$28,676.47","Promedio":"$19,020.13"},"3":{"segmento":"D","M\\u00ednimo":"$1,221.80","M\\u00e1ximo":"$23,913.04","Promedio":"$15,739.80"},"4":{"segmento":"E","M\\u00ednimo":"$651.04","M\\u00e1ximo":"$53,571.43","Promedio":"$80,156.97"},"5":{"segmento":"L","M\\u00ednimo":"$237.50","M\\u00e1ximo":"$45,333.33","Promedio":"$32,169.16"},"6":{"segmento":"S","M\\u00ednimo":"$464.20","M\\u00e1ximo":"$39,062.50","Promedio":"$23,807.25"},"7":{"segmento":"general","M\\u00ednimo":"$164.11","M\\u00e1ximo":"$39,669.42","Promedio":"$27,505.91"}}'

## M2 Terreno

In [22]:
df_total = df_total[df_total['metros_construido'] > 40]
df_total_terreno = df_total[~df_total['metros_total'].isin([0, 1, 2])]

df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe(percentiles=[0.96]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '96%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    'min': 'Mínimo',
    '96%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     30.00 m²
1     52.00 m²
2     42.00 m²
3     44.00 m²
4     35.00 m²
5    103.00 m²
6     42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1393368157.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      292.00 m²
1      201.00 m²
2      198.00 m²
3      279.48 m²
4    1,640.00 m²
5    2,200.00 m²
6      600.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(

,segmento,Mínimo,Máximo,Promedio
0,A,30.00 m²,292.00 m²,168.82 m²
1,B,52.00 m²,201.00 m²,129.77 m²
2,C,42.00 m²,198.00 m²,123.72 m²
3,D,44.00 m²,279.48 m²,114.14 m²
4,E,35.00 m²,"1,640.00 m²",698.24 m²
5,L,103.00 m²,"2,200.00 m²",885.85 m²
6,S,42.00 m²,600.00 m²,522.45 m²


In [24]:
todo_descripcion = df_total[~df_total['metros_total'].isin([0, 1, 2])]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    'min': 'Mínimo',
    '95%': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1320778984.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    30.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1320778984.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    780.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1320778984.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,50%,segmento
metros_total,30.00 m²,780.00 m²,419.84 m²,189.00 m²,general


In [25]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [26]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,50%
0,A,30.00 m²,292.00 m²,168.82 m²,NaN
1,B,52.00 m²,201.00 m²,129.77 m²,NaN
2,C,42.00 m²,198.00 m²,123.72 m²,NaN
3,D,44.00 m²,279.48 m²,114.14 m²,NaN
4,E,35.00 m²,"1,640.00 m²",698.24 m²,NaN
5,L,103.00 m²,"2,200.00 m²",885.85 m²,NaN
6,S,42.00 m²,600.00 m²,522.45 m²,NaN
7,general,30.00 m²,780.00 m²,419.84 m²,189.00 m²


## M2 Construcción

In [27]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe(percentiles=[0.95]).reset_index()
df_descripcion = df_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
df_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     51.00 m²
1     52.00 m²
2     42.00 m²
3     42.00 m²
4     41.00 m²
5    165.00 m²
6     42.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\1810280376.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      254.00 m²
1      202.00 m²
2      211.00 m²
3      276.00 m²
4    1,000.00 m²
5      865.00 m²
6      456.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(

,segmento,Mínimo,Máximo,Promedio
0,A,51.00 m²,254.00 m²,180.02 m²
1,B,52.00 m²,202.00 m²,137.56 m²
2,C,42.00 m²,211.00 m²,129.46 m²
3,D,42.00 m²,276.00 m²,113.19 m²
4,E,41.00 m²,"1,000.00 m²",506.25 m²
5,L,165.00 m²,865.00 m²,636.52 m²
6,S,42.00 m²,456.00 m²,274.32 m²


In [29]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0,1,2,3]) & (df_total['metros_total'] > 3)]['metros_total'].describe(percentiles=[0.95])[['min', '95%', 'mean',]].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(3):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Máximo Total', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    30.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    780.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\2220166576.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

,Mínimo,Máximo,Promedio,segmento
metros_total,30.00 m²,780.00 m²,419.84 m²,general


In [30]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finaleajs_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [31]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,51.00 m²,254.00 m²,180.02 m²
1,B,52.00 m²,202.00 m²,137.56 m²
2,C,42.00 m²,211.00 m²,129.46 m²
3,D,42.00 m²,276.00 m²,113.19 m²
4,E,41.00 m²,"1,000.00 m²",506.25 m²
5,L,165.00 m²,865.00 m²,636.52 m²
6,S,42.00 m²,456.00 m²,274.32 m²
7,general,30.00 m²,780.00 m²,419.84 m²


## Pecio M2 Construcción

In [32]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe(percentiles=[0.95]).reset_index()
# Mantener solo las columnas que deseas (en este caso, min, 95%, mean)
grupo_filtrado = grupo_filtrado[['segmento', 'min', '95%', 'mean']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $169.53
1    $5,930.02
2      $164.11
3    $1,221.80
4      $651.04
5      $237.50
6      $464.20
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\952371004.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    $37,563.22
1    $35,119.05
2    $28,676.47
3    $23,913.04
4    $53,571.43
5    $45,333.33
6    $39,062.50
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(format

,segmento,Mínimo,Máximo,Promedio
0,A,$169.53,"$37,563.22","$23,359.40"
1,B,"$5,930.02","$35,119.05","$22,788.10"
2,C,$164.11,"$28,676.47","$19,020.13"
3,D,"$1,221.80","$23,913.04","$15,739.80"
4,E,$651.04,"$53,571.43","$80,156.97"
5,L,$237.50,"$45,333.33","$32,169.16"
6,S,$464.20,"$39,062.50","$23,807.25"


In [33]:
todo_descripcion = df_total[(df_total['precio_m2_construido'] > 6) & (~df_total['metros_construido'].isin([0, 1,2,3]))]['precio_m2_construido'].describe(percentiles=[0.95])[['min', '95%', 'mean']].to_frame().T
todo_descripcion['segmento'] = 'general'
todo_descripcion = todo_descripcion[['segmento', 'min', '95%', 'mean']] #Columnas deseadas

todo_descripcion.rename(columns={
    '95%': 'Máximo',
    'min': 'Mínimo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 4):  # 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $164.11
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $39,669.42
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_15004\4241514475.py:13: FutureWarning: Setting an item of incompatible dtype is dep

,segmento,Mínimo,Máximo,Promedio
precio_m2_construido,general,$164.11,"$39,669.42","$27,505.91"


In [34]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [35]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio
0,A,$169.53,"$37,563.22","$23,359.40"
1,B,"$5,930.02","$35,119.05","$22,788.10"
2,C,$164.11,"$28,676.47","$19,020.13"
3,D,"$1,221.80","$23,913.04","$15,739.80"
4,E,$651.04,"$53,571.43","$80,156.97"
5,L,$237.50,"$45,333.33","$32,169.16"
6,S,$464.20,"$39,062.50","$23,807.25"
7,general,$164.11,"$39,669.42","$27,505.91"
